In [1]:
import sqlite3
import pandas as pd

# Create in-memory DB
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()


In [2]:
cursor.executescript("""
CREATE TABLE customers (
    customer_id INT PRIMARY KEY,
    customer_name TEXT,
    signup_date DATE
);

CREATE TABLE orders (
    order_id INT PRIMARY KEY,
    customer_id INT,
    order_date DATE,
    status TEXT,
    total_amount DECIMAL(10,2),
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
);

CREATE TABLE order_items (
    item_id INT PRIMARY KEY,
    order_id INT,
    product_name TEXT,
    quantity INT,
    price DECIMAL(10,2),
    FOREIGN KEY (order_id) REFERENCES orders(order_id)
);
""")


In [3]:
cursor.executescript("""
INSERT INTO customers VALUES (1, 'Alice', '2024-01-10');
INSERT INTO customers VALUES (2, 'Bob', '2024-01-15');
INSERT INTO customers VALUES (3, 'Charlie', '2024-01-20');

INSERT INTO orders VALUES (101, 1, '2024-02-01', 'Completed', 250.00);
INSERT INTO orders VALUES (102, 1, '2024-03-01', 'Completed', 300.00);
INSERT INTO orders VALUES (103, 2, '2024-02-05', 'Completed', 150.00);

INSERT INTO order_items VALUES (1, 101, 'Shoes', 1, 250.00);
INSERT INTO order_items VALUES (2, 102, 'Bag', 1, 300.00);
INSERT INTO order_items VALUES (3, 103, 'Shoes', 1, 150.00);
""")


In [4]:
query = """
SELECT c.customer_name, SUM(o.total_amount) AS total_spent
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
GROUP BY c.customer_name
ORDER BY total_spent DESC;
"""
pd.read_sql_query(query, conn)


,customer_name,total_spent
0,Alice,550
1,Bob,150


In [5]:
query = """
SELECT
    strftime('%Y-%m', order_date) AS month,
    SUM(total_amount) AS total_revenue
FROM orders
GROUP BY month
ORDER BY month;
"""
pd.read_sql_query(query, conn)


,month,total_revenue
0,2024-02,400
1,2024-03,300


In [6]:
query = """
SELECT
    product_name,
    SUM(quantity) AS total_quantity_sold
FROM order_items
GROUP BY product_name
ORDER BY total_quantity_sold DESC
LIMIT 5;
"""
pd.read_sql_query(query, conn)


,product_name,total_quantity_sold
0,Shoes,2
1,Bag,1


In [7]:
query = """
SELECT
    ROUND(AVG(total_amount), 2) AS avg_order_value
FROM orders;
"""
pd.read_sql_query(query, conn)


,avg_order_value
0,233.33


In [8]:
query = """
WITH recent_orders AS (
    SELECT customer_id, MAX(order_date) AS last_order_date
    FROM orders
    GROUP BY customer_id
)
SELECT
    COUNT(CASE WHEN last_order_date >= DATE('now', '-30 day') THEN 1 END) AS active_customers,
    COUNT(CASE WHEN last_order_date < DATE('now', '-30 day') THEN 1 END) AS inactive_customers
FROM recent_orders;
"""
pd.read_sql_query(query, conn)


,active_customers,inactive_customers
0,0,2
